In [1]:
#Importamos las librerias necesarias
import requests
import re
from bs4 import BeautifulSoup

In [2]:
# Setear el header
head = {'user-agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
            ' Chrome/61.0.3163.100 Safari/537.36'}

In [3]:
canales = [
"/lo-ultimo/",
"/politica/",
"/mundo/",
"/sociedad/",
"/policiales/",
"/ciudades/",
"/opinion/",
"/cartas_al_pais/",
"/cultura/",
"/rural/",
"economia/",
"/tecnologia/",
"/revista-enie/",
"/viva/",
"/br/",
"/deportes/futbol/",
"/deportes/ascenso/",
"/deportes/futbol-internacional/",
"/deportes/messi/",
"/deportes/rugby/",
"/deportes/tenis/",
"/deportes/automovilismo/",
"/deportes/basquet/",
"/deportes/polideportivo/",
"/deportes/turf/",
"/deportes/boxeo/",
"/deportes/hockey/",
"/deportes/",
"/espectaculos/fama/",
"/espectaculos/tv/",
"/espectaculos/cine/",
"/espectaculos/musica/",
"/espectaculos/teatro/",
"/si/",
"/espectaculos/",
"/entremujeres/",
"/autos/",
"/buena-vida/",
"/viajes/",
"/arq/"
]

In [4]:
def scrapearCanal(canal):
    url = 'https://www.clarin.com/rss' + canal
    resp = requests.get(url, headers = head)
    soup = BeautifulSoup(resp.text, 'xml')
    noticias = scrapearSoup(soup, canal)
    return noticias
    

In [5]:
def scrapearSoup(soup, canal):
    items = soup.findAll('item')
    url_nots = []
    for item in items:
         url_nots += item.findAll('link')    
    noticias = []
    for url_not in url_nots:
        noticias.append(parsearNoticia(url_not.text, canal))
    return(noticias)

In [6]:
def parsearNoticia(url, canal):
    try:
        noticia = {}
        resp = requests.get(url, headers = head)
        soup_noticia = BeautifulSoup(resp.text, 'html.parser')
        # obtener la fecha
        noticia['url'] = url
        # obtener el cuerpo
        try:
            noticia['cuerpo'] = soup_noticia.findAll('div',{'class':'body-nota'})[0].get_text()
        except:
            noticia['cuerpo'] = ''
        try:
            noticia['fecha_hora'] = soup_noticia.findAll('span',{'data-momento-format':'fecha_hora'})[0].get_text()
        except:
            noticia['fecha_hora'] = ''
        # título
        try:
            noticia['titulo'] = soup_noticia.findAll('h1',{'id':'title'})[0].get_text()
        except:
            noticia['titulo'] = ''
        # resumen
        try:
            noticia['resumen'] = soup_noticia.findAll('div',{'itemprop':'description'})[0].get_text()
        except:
            noticia['resumen'] = ''
        # imagen
        try:
            noticia['imagen'] = soup_noticia.findAll('img',{'class':'img-responsive intelResolution'})[0].get('data-big')
        except:
            noticia['imagen'] = ''
        return noticia
        noticia['suplemento'] = canal
    except:
        return {'url':url,'cuerpo':'','fecha_hora':'','titulo':'','resumen':'','imagen':'', 'suplemento':canal}
    

In [7]:
noticias = []
for i in canales:
    noticias += scrapearCanal(i)

In [8]:
len(noticias)

372

In [9]:
type(noticias[0])

dict

In [10]:
import pandas as pd
df = pd.DataFrame(noticias)

In [12]:
df.shape

(372, 6)

In [15]:
import time
df.to_csv('clarin' + time.strftime("%d-%m-%Y") + '.csv',encoding='utf-8')

In [16]:
df.shape

(372, 6)